In [2]:
# dash pie all sites
from dash import Dash, html, dcc
import pandas as pd
import plotly.express as px
import os

# Load data
CSV_PATH = r"C:\Users\whas3\Desktop\Data World\My projects\I-C\spacex_launch_dash.csv"
df = pd.read_csv(CSV_PATH)

#Column aliases & cleaning
def pick(cols, *cands):
    for c in cands:
        if c in cols: return c
    raise KeyError(f"Missing columns: tried {cands}, found {list(cols)}")

SITE_COL = pick(df.columns, "Launch Site", "LaunchSite")
CLASS_COL = pick(df.columns, "class", "Class", "Outcome", "Success")

df[CLASS_COL] = pd.to_numeric(df[CLASS_COL], errors="coerce").fillna(0).astype(int)

#Success counts by site
success_counts = (
    df[df[CLASS_COL] == 1]
      .groupby(SITE_COL)
      .size()
      .reset_index(name="Successes")
      .sort_values("Successes", ascending=False)
)

#Figure
fig = px.pie(
    success_counts,
    values="Successes",
    names=SITE_COL,
    title="Launch Success Count by Site (All Sites)",
    hole=0.25
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.update_layout(margin=dict(l=30, r=30, t=60, b=30))



#Dash app
app = Dash(__name__)
app.title = "SpaceX Success Pie"
app.layout = html.Div(
    style={"maxWidth": "900px", "margin": "40px auto", "fontFamily": "system-ui, sans-serif"},
    children=[
        html.H2("Launch Success Count for All Sites"),
        dcc.Graph(id="success-pie-chart", figure=fig),
        html.Div("Data source: spacex_launch_dash.csv", style={"color": "#666", "fontSize": "12px"})
    ]
)

if __name__ == "__main__":
    #Run
    app.run(debug=True, port=8050)
